## 中文评估


In [55]:
file_path = "../.data/all_raretrible.txt"
data = open(file_path, "r", encoding="utf-8").read().split("\n")
data = [x.strip().split(",") for x in data if len(x) > 0]
data = [[x[0], ",".join(x[1:])] for x in data]
print(data[0:2])

[['感激', '好的,那我谢谢您了。'], ['中性', '没有词汇课的优惠券吗?']]


In [72]:
import pandas as pd

columns=["label", "text",]
# 转换 data 到 pandas DataFrame
df = pd.DataFrame(data, columns=columns)


In [73]:
from dspy.datasets import DataLoader

# 创建 dspy.datasets 对象
dl = DataLoader()
# fields 选择指定列 input_keys
all_dataset = dl.from_pandas(
    df,
    fields=columns,
    input_keys=("text",),
)
print(all_dataset[0:2])

[Example({'label': '感激', 'text': '好的,那我谢谢您了。'}) (input_keys={'text'}), Example({'label': '中性', 'text': '没有词汇课的优惠券吗?'}) (input_keys={'text'})]


In [74]:
from collections import defaultdict
import random


def stratified_sample(from_dataset, label_attr, *dataset_sizes):
    label_groups = defaultdict(list)
    for item in from_dataset:
        label = getattr(item, label_attr)
        label_groups[label].append(item)

    result_sets = [[] for _ in dataset_sizes]

    for label, items in label_groups.items():
        random.shuffle(items)
        label_sizes = [size // len(label_groups) for size in dataset_sizes]
        start = 0
        for i, size in enumerate(label_sizes):
            end = start + size
            result_sets[i].extend(items[start:end])
            start = end

    all_items = [item for items in label_groups.values() for item in items]
    for i, (result_set, target_size) in enumerate(zip(result_sets, dataset_sizes)):
        shortage = target_size - len(result_set)
        if shortage > 0:
            result_sets[i].extend(random.sample(all_items, shortage))

    return result_sets


In [75]:
train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)
from collections import Counter

train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)
for dataset in [train_set, test_set, sample_set]:
    label_counts = Counter(item.get("label") for item in dataset)
    print(label_counts)

Counter({'感激': 3, '中性': 3, '焦急': 3, '惊讶': 3, '高兴': 3, '抱怨': 3, '生气': 2})
Counter({'生气': 8, '感激': 7, '中性': 7, '焦急': 7, '惊讶': 7, '高兴': 7, '抱怨': 7})
Counter({'感激': 16, '中性': 14, '焦急': 14, '惊讶': 14, '高兴': 14, '抱怨': 14, '生气': 14})


In [104]:
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set)
ret = ana.do()
print(ret)

d:\dev\py-venv\qa\Lib\site-packages\pydantic\_internal\_generate_schema.py:404: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


You are a customer service quality inspection expert., sentiment analysis from Question

---

Follow the following format.

Question: list of input text
Answer:
    Return format: a list of "Index, Emotion Category, Emotion Subcategory, Emotion Subcategory Score"
    Emotion Subcategory Score (0-1.00).
    Example: 
        1, pessimistic, complaining, 0.90
        2, optimistic, happy, 0.75
Please choose emotion categories and subcategories only from the following options:
    Emotion categories: ["pessimistic", "optimistic", "neutral"]
    Emotion subcategories: ['neutral', 'surprised', 'thankful', 'complaining', 'urgent','anxious', 'angry', 'happy']
    
Here is some examples
---

Question: 
1: 好的谢谢!!
Answer:
1,neutral,thankful,0.85

---

Question:
1: 谢了亲电话打过了
Answer:
1,optimistic,thankful,0.85

---

Question:
1: 我看下都吓我一跳
Answer:
1,pessimistic,surprised,0.90

---

Question:
1: 哈哈哈哈可以的亲
Answer:
1,optimistic,happy,0.80

---

Question:
```
1: 好的谢谢[亲]
2: 弄好了,谢谢了
3: 好的,谢谢您的提示/:@)
4: 好的,设

In [105]:
ana.show_error()

> 1, neutral, thankful, 0.85   Example({'label': '感激', 'text': '好的谢谢[亲]', 'pred_label': None, 'main': None, 'score': ' 0.75'}) (input_keys={'text'})
> 2, optimistic, thankful, 0.85   Example({'label': '感激', 'text': '弄好了,谢谢了', 'pred_label': None, 'main': None, 'score': ' 0.75'}) (input_keys={'text'})
> 3, neutral, thankful, 0.85   Example({'label': '感激', 'text': '好的,谢谢您的提示/:@)', 'pred_label': None, 'main': None, 'score': ' 0.75'}) (input_keys={'text'})
> 4, neutral, thankful, 0.85   Example({'label': '感激', 'text': '好的,设置出问题,现在好了,谢谢', 'pred_label': None, 'main': None, 'score': ' 0.75'}) (input_keys={'text'})
> 5, neutral, thankful, 0.85   Example({'label': '感激', 'text': '我知道了,谢谢你!', 'pred_label': None, 'main': None, 'score': ' 0.75'}) (input_keys={'text'})
> 6, neutral, urgent, 0.90   Example({'label': '感激', 'text': '我得走了,请尽快处理,谢谢', 'pred_label': None, 'main': None, 'score': ' 0.85'}) (input_keys={'text'})
> 7, neutral, thankful, 0.85   Example({'label': '感激', 'text': '好的谢谢你,新年快乐哈', 'pre

In [93]:
print(ana.respone.content)

Answer:
1, neutral, thankful, 0.85
2, neutral, thankful, 0.85
3, neutral, thankful, 0.85
4, neutral, thankful, 0.85
5, neutral, thankful, 0.85
6, neutral, anxious, 0.75
7, neutral, thankful, 0.85
8, neutral, neutral, 0.60
9, neutral, neutral, 0.60
10, neutral, neutral, 0.60
11, neutral, neutral, 0.60
12, neutral, neutral, 0.60
13, neutral, neutral, 0.60
14, neutral, neutral, 0.60
15, neutral, urgent, 0.85
16, neutral, urgent, 0.85
17, neutral, urgent, 0.85
18, neutral, urgent, 0.85
19, neutral, neutral, 0.60
20, neutral, urgent, 0.85
21, neutral, urgent, 0.85
22, neutral, neutral, 0.60
23, neutral, neutral, 0.60
24, neutral, neutral, 0.60
25, neutral, neutral, 0.60
26, neutral, neutral, 0.60
27, neutral, neutral, 0.60
28, optimistic, happy, 0.80
29, neutral, neutral, 0.60
30, neutral, neutral, 0.60
31, neutral, thankful, 0.85
32, neutral, happy, 0.80
33, neutral, neutral, 0.60
34, neutral, neutral, 0.60
35, neutral, neutral, 0.60
36, neutral, neutral, 0.60
37, neutral, neutral, 0.60
38

In [86]:

main_cover = analysis.LangCover(
        en=["pessimistic", "optimistic", "neutral"],
        cn=["负面", "正面", "中性"],
    )
for word in ["pessimistic", "optimistic", "neutral"]:
    print(main_cover.to_cn(word))

负面
正面
中性


# 测试印尼语


In [36]:
file_path = "../.data/all_raretrible.txt"
# file_path = "../.data/mix_model/data/all_raretrible.txt"
# file_path = "/home/azhao/projects/ai/train/liubin/mix_model/data/all_raretrible.txt"
data = open(file_path, "r", encoding="utf-8").read().split("\n")
data = [x.strip().split(",") for x in data if len(x) > 0]
data = [[x[0], ",".join(x[1:]), "", "", "", ""] for x in data]
print(data[0:2])

[['感激', '好的,那我谢谢您了。', '', '', '', ''], ['中性', '没有词汇课的优惠券吗?', '', '', '', '']]


In [37]:
import pandas as pd

columns=["label", "text", "pred_label","main", "score", "keywords"]
# 转换 data 到 pandas DataFrame
df = pd.DataFrame(data, columns=columns)

# 保存为 CSV 文件
cvs_file = "../.data/data.csv"
df.to_csv(cvs_file, index=False)

In [38]:

from dspy.datasets import DataLoader

# 创建 dspy.datasets 对象
dl = DataLoader()
# fields 选择指定列 input_keys
all_dataset = dl.from_csv(
    cvs_file,
    fields=columns,
    input_keys=("text",),
)
print(all_dataset[0:2])

Generating train split: 0 examples [00:00, ? examples/s]

[Example({'label': '感激', 'text': '好的,那我谢谢您了。', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'}), Example({'label': '中性', 'text': '没有词汇课的优惠券吗?', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})]


In [39]:
from collections import defaultdict
import random


def stratified_sample(from_dataset, label_attr, *dataset_sizes):
    label_groups = defaultdict(list)
    for item in from_dataset:
        label = getattr(item, label_attr)
        label_groups[label].append(item)

    result_sets = [[] for _ in dataset_sizes]

    for label, items in label_groups.items():
        random.shuffle(items)
        label_sizes = [size // len(label_groups) for size in dataset_sizes]
        start = 0
        for i, size in enumerate(label_sizes):
            end = start + size
            result_sets[i].extend(items[start:end])
            start = end

    all_items = [item for items in label_groups.values() for item in items]
    for i, (result_set, target_size) in enumerate(zip(result_sets, dataset_sizes)):
        shortage = target_size - len(result_set)
        if shortage > 0:
            result_sets[i].extend(random.sample(all_items, shortage))

    return result_sets


In [40]:
# 使用示例
train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)

In [41]:
from collections import Counter

train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)
for dataset in [train_set, test_set, sample_set]:
    label_counts = Counter(item.get("label") for item in dataset)
    print(label_counts)

Counter({'感激': 4, '惊讶': 4, '中性': 3, '抱怨': 3, '焦急': 2, '高兴': 2, '生气': 2})
Counter({'高兴': 8, '感激': 7, '中性': 7, '焦急': 7, '惊讶': 7, '抱怨': 7, '生气': 7})
Counter({'中性': 15, '高兴': 15, '感激': 14, '焦急': 14, '惊讶': 14, '抱怨': 14, '生气': 14})


In [42]:
import random

random.shuffle(test_set)
for item in test_set:
    print(item)

Example({'label': '抱怨', 'text': '很臭', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '抱怨', 'text': '每次找你们都这样那样', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '生气', 'text': '凭什么收益不让取,那也是我的钱。你们是按天发利息的,既然发给我了,就是我的钱,凭什么不能取', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '焦急', 'text': '很急[emoji040]', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '惊讶', 'text': '这个还扣我费用啊', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '感激', 'text': '谢谢了,我的问题已经解决了', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '感激', 'text': '好,谢~', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '抱怨', 

In [44]:
import importlib
import analysis

importlib.reload(analysis)

messages = []
for i, item in enumerate(test_set):
    messages.append(item.get('text'))
ana = analysis.Analysis(messages=messages)
ret = ana.do()
print(ret.content)

You are a customer service quality inspection expert., sentiment analysis from Question

---

Follow the following format.

Question: list of input text
Answer: 
    返回格式 "序号,情绪分类,情绪子分类,情绪分类评分" 
    情绪分类包括 正向,负向,中性
    情绪子分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']，
    情绪子分类评分(0-1.0)。
    例如 1,负向,抱怨,0.9
注意：请严格上面的分类来输出，不要输出不在情绪子分类包括中的的分类

Here is an example
---

Question: 好的谢谢!!
Answer: 1,正向,感激,0.8

---

Question: 谢了亲电话打过了
Answer: 2,正向,感激,0.8

---

Question: 我看下都吓我一跳
Answer: 3,负向,惊讶,0.9

---

Question: 哈哈哈哈可以的亲
Answer: 4,正向,高兴,0.8

---

Question:
```
0: 很臭
1: 每次找你们都这样那样
2: 凭什么收益不让取,那也是我的钱。你们是按天发利息的,既然发给我了,就是我的钱,凭什么不能取
3: 很急[emoji040]
4: 这个还扣我费用啊
5: 谢谢了,我的问题已经解决了
6: 好,谢~
7: 你们这个不行没人家的好
8: 这名字怎么这么可爱
9: 流程还没走完?
10: 抓紧时间发货,做点实事,别总嗯嗯。
11: 还要收费???
12: 移动公司都没有让我进黑名单,这么小破公司无缘无故让我成为黑名单,你那边应该有记录才对,我为啥成为黑名单?
13: 我现在是想投诉你们在线客服
14: 今天鸡这么晚才送来
15: 太期待了
16: 爱你呦
17: 仓没满换还不了?
18: 我已经说了好多次了,都没帮我解决
19: 加油加油加油亲爱的
20: 妹妹先告诉我查件结果
21: 搞不懂这些**要来干嘛
22: 要是我记不住火锅店那我不亏死了?
23: 收到,谢谢!正常要多久才会换货过来?
24: 可以,我换其他的车
2

In [45]:
# 答案和问题对应起来
def update_sample(sample_set, answer):
    # 删除 answer 中的 'Answer: ', 删除空行
    answer = answer.replace("Answer: ", "").strip()
    for sample, prediction in zip(sample_set, answer.split("\n")):
        sample["score"] = prediction.split(",")[3]
        sample["pred_label"] = prediction.split(",")[2]
        sample["main"] = prediction.split(",")[1]


def sample_error_rate(sample_set):
    incorrect_count = sum(
        1 for sample in sample_set if sample["pred_label"] != sample["label"]
    )
    total_count = len(sample_set)
    return incorrect_count / total_count

In [46]:
update_sample(test_set, ret.content)
print(sample_error_rate(test_set))


0.84


In [35]:
idx = 0
for sample in test_set:
    if sample["pred_label"] != sample["label"]:
        print(f"Question {idx}: {sample}")
        idx+=1

Question 0: Example({'label': '焦急', 'text': '好的,麻烦尽快,谢谢、', 'pred_label': '感激', 'main': '正向', 'score': '0.8', 'keywords': '正向'}) (input_keys={'text'})
Question 1: Example({'label': '焦急', 'text': '你们发票开的这么慢吗?', 'pred_label': '抱怨', 'main': '负向', 'score': '0.8', 'keywords': '负向'}) (input_keys={'text'})
Question 2: Example({'label': '高兴', 'text': '太期待了', 'pred_label': '期待', 'main': '正向', 'score': '0.8', 'keywords': '正向'}) (input_keys={'text'})
Question 3: Example({'label': '惊讶', 'text': '我昨天买的今天不是应该确认吗', 'pred_label': '催促', 'main': '中性', 'score': '0.7', 'keywords': '负向'}) (input_keys={'text'})
Question 4: Example({'label': '感激', 'text': '好的,明白,谢谢你', 'pred_label': '感谢', 'main': '中性', 'score': '0.8', 'keywords': '正向'}) (input_keys={'text'})
Question 5: Example({'label': '惊讶', 'text': '只是协商嘛,不是退吗?你刚才不是说退吗?', 'pred_label': '疑问', 'main': '中性', 'score': '0.7', 'keywords': '负向'}) (input_keys={'text'})
Question 6: Example({'label': '中性', 'text': '我现在去你们的归还点都没有你们的机器', 'pred_label': '抱怨', 'main': '负向